In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install -U bitsandbytes faiss-cpu sentence-transformers transformers torch pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 433.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

import pandas as pd
import numpy as np
import faiss
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import logging

logging.basicConfig(filename='medical_queries.log', level=logging.INFO, format='%(asctime)s %(message)s')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model Configuration
model_name = "BioMistral/BioMistral-7B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
).to(device)


`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [ ]:

# Dataset Preparation
df1 = pd.read_json("hf://datasets/FreedomIntelligence/medical-o1-reasoning-SFT/medical_o1_sft.json")

def row_to_text1(row):
    return f"Question: {row['Question']}\nContext: {row['Complex_CoT']}\nAnswer: {row['Response']}"


df2 = pd.read_csv("train.csv")

def row_to_text2(row):
    return f"Question: {row['Question']}\nAnswer: {row['Answer']}"


texts = df.apply(row_to_text, axis=1).tolist() + df2.apply(row_to_text2, axis=1).tolist()



In [ ]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(texts, convert_to_tensor=False)
embeddings = np.array([emb / np.linalg.norm(emb) for emb in embeddings])

embedding_dim = embeddings.shape[1]
quantizer = faiss.IndexFlatIP(embedding_dim)
index = faiss.IndexIVFFlat(quantizer, embedding_dim, 100, faiss.METRIC_INNER_PRODUCT)
index.train(embeddings)
index.add(embeddings)

def get_context(question, top_k=3, threshold=0.7):
    q_emb = embedder.encode(question, convert_to_tensor=False)
    q_emb /= np.linalg.norm(q_emb)
    scores, idx = index.search(np.array([q_emb]), top_k)
    relevant_texts = [texts[i] for i, score in zip(idx[0], scores[0]) if score >= threshold]
    return "\n\n".join(relevant_texts), scores

In [ ]:
model = torch.compile(model)

In [ ]:
chat_history = []

def generate_answer(question, chat_history=None, max_length=512):
    context, scores = get_context(question)

    prompt_intro = (
        "You are a licensed medical doctor providing accurate, helpful, and professional answers to medical questions.\n"
        "Speak clearly, like you would to a patient—neither too robotic nor overly casual.\n"
        "Use bullet points when listing symptoms, treatments, or steps.\n"
        "Break long explanations into clear, readable paragraphs with line breaks.\n"
        "Be concise when the answer is simple. Be detailed only when necessary.\n"
        "Structure your response using sections like Overview, Symptoms, Treatment, and Recovery if relevant."
    )

    formatted_history = "\n".join([f"User: {q}\nDoctor: {a}" for q, a in chat_history]) if chat_history else ""

    if len(context.strip()) == 0:
        prompt = f"""{prompt_intro}\n\nConversational History (for context awareness):\n{formatted_history}\n\nQuestion: {question}\n\nDoctor's Answer:"""
    else:
        prompt = f"""{prompt_intro}\n\nConversational History (for context awareness):\n{formatted_history}\n\nContext (for awareness only, do not repeat it):\n{context}\n\nQuestion: {question}\n\nDoctor's Answer:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=max_length,
            temperature=0.3,
            top_k=30,
            top_p=0.8,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id  # 🔁 cleaner cutoff
        )

    response = tokenizer.decode(outputs[0].cpu(), skip_special_tokens=True)
    answer = response.split("Doctor's Answer:")[-1].strip().split("Question:")[0].strip()  # 🔁 safer slicing

    logging.info(f"Question: {question}\nAnswer: {answer}\nSimilarity Scores: {scores}")
    return answer


In [ ]:
question = "side effect of diabetics, give it in points"
answer = generate_answer(question, chat_history)
chat_history.append((question, answer))
chat_history[-6:]
print(answer)

Diabetes is a chronic disease that affects the way the body processes blood sugar. It can lead to serious health problems, such as heart disease, stroke, and kidney failure. The most common side effect of diabetes is hypoglycemia, which is a drop in blood sugar levels. This can cause symptoms such as dizziness, sweating, and confusion. Other side effects of diabetes include weight gain or loss, increased thirst and urination, and blurred vision. It is important for people with diabetes to work closely with their healthcare provider to manage their blood sugar levels and prevent these side effects.


In [ ]:
!pip install pyngrok flask_cors flask

In [ ]:
# This block sets up a Flask API for serving the multi-agent LLM system.
# It handles POST requests at `/chat`, invokes the query handler, and returns friendly, contextual responses.
# Ngrok is used to expose the local Flask server to the internet for quick testing and integration.

from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading
from flask_cors import CORS

ngrok.set_auth_token("2vKd5NUrpcGWaFsdZasMC33LmKz_4gFCRuo9sKEeP8bQPjLnA")

# Initialize Flask app and enable CORS
app = Flask(__name__)
CORS(app)

# Basic health check route
@app.route("/")
def home():
    return jsonify({"message": "Flask app is live on ngrok !"})

# Main chat endpoint: receives user question and returns LLM-based answer
@app.route("/chat", methods=["POST"])
def chat():
    data = request.get_json()
    question = data.get("question", "")

    answer = generate_answer(question, chat_history)
    chat_history.append((question, answer))
    chat_history[-6:]

    return jsonify({"reply": answer})

# Run the Flask app in a background thread
def run():
    app.run(port=5005, debug=True, use_reloader=False, threaded=True)

threading.Thread(target=run).start()

# Open a public ngrok tunnel to expose port 5004
public_url = ngrok.connect(5005)
print("Ngrok Tunnel URL:", public_url)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5005
INFO:werkzeug:Press CTRL+C to quit


Ngrok Tunnel URL: NgrokTunnel: "https://e2b4-34-16-202-33.ngrok-free.app" -> "http://localhost:5005"


In [ ]:
The symptoms of diabetes include increased thirst, increased hunger, increased urination, weight loss, fatigue, blurred vision, slow healing sores, and frequent infections.

SyntaxError: invalid syntax (<ipython-input-48-15053572a37f>, line 1)